In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import json


In [2]:
#create a dataframe from a file using pandas
file = '../Resources/Capitals.csv'
# The correct encoding must be used to read the CSV in pandas
weather = pd.read_csv(file, encoding="ISO-8859-1")
weather.reset_index(inplace=True)
weather.columns

Index(['index', 'Country', 'Cities', 'Country Code'], dtype='object')

In [3]:
# set up additional columns to hold information
weather['Temperature'] = ""

base_url = "http://api.openweathermap.org/data/2.5/weather?"
#url = base_url + "appid=" + "082ff2cde1d396b2b896b2335daad199" + "&q=" + city
city = 'Minneapolis'

# use iterrows to iterate through pandas dataframe
for index, row in weather.iterrows():

    # iterate thru each row of the file
    city = row['Cities']
    url = base_url + "appid=" + "082ff2cde1d396b2b896b2335daad199" + "&q=" + city
    
#     # update Param1 from params dict
#     params['Param1'] = VariableFOrParam1
#     # add param3 to params dic
#     params['Param3'] = FromWhereverRowVAriableOrother
# assemble url and make API request
    response = requests.get(url).json()
    
    # extract results
    #print(response['main']['temp'])
    
    #update the dataframe 
    try:
        weather.loc[index, 'Temperature'] = response['main']['temp']
        
    #except (KeyError, IndexError):
     #   print("Missing field/result... skipping.")

SyntaxError: unexpected EOF while parsing (<ipython-input-3-deb893a57fba>, line 30)

In [4]:
weather

,index,Country,Cities,Country Code
0,0,Afghanistan,Kabul,AF
1,1,Albania,Tirana (Tirane),AL
2,2,Algeria,Algiers,DZ
3,3,Andorra,Andorra la Vella,AD
4,4,Angola,Luanda,AO
5,5,Antigua and Barbuda,Saint John's,AG
6,6,Argentina,Buenos Aires,AR
7,7,Armenia,Yerevan,AM
8,8,Australia,Canberra,AU
9,9,Austria,Vienna,AT


In [5]:
weather.to_csv('../Outputs/WeatherbyCity.csv')